In [ ]:
!pip install anthropic pandas jupyter openai

In [ ]:
import random
import openai
import pandas as pd
import os
import glob
import time
from google.colab import userdata, drive

In [ ]:
# Mount Google Drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
# Set up OpenAI API key
openai_api_key = userdata.get('OPENAI_API_KEY')
if openai_api_key is None or openai_api_key.strip() == "":
    raise ValueError("OpenAI API key is not set. Please set it in Colab secrets.")
openai.api_key = openai_api_key
print("OpenAI API key set successfully.")

OpenAI API key set successfully.


In [ ]:
# def generate_description_and_prompt(test_case_name, test_case_type, file_path):
#     parts = file_path.split('/')
#     application_name = parts[-3]
#     directory_name = parts[-2]

#     # Dynamic action and type based on test case type
#     actions = ['Generate', 'Create', 'Build', 'Validate']
#     action = random.choice(actions)
#     type_phrase = 'keyword-driven' if test_case_type == 'keyword-driven' else 'data-driven'

#     # Generate a dynamic prompt
#     prompt = f"{action} a robot framework test case for {type_phrase} testing of {directory_name.replace('_', ' ')} functionality in the {application_name} platform, specifically {test_case_name}."

#     description_template = f"Provide a high-level description for the test case '{test_case_name}' including its purpose and key objectives."

#     try:
#         response = openai.ChatCompletion.create(
#             model="gpt-4o",
#             messages=[{"role": "user", "content": description_template}],
#             max_tokens=150
#         )
#         description = response.choices[0].message['content'].strip()
#     except Exception as e:
#         print(f"Error generating description for {test_case_name}: {str(e)}")
#         description = f"Description for {test_case_name} could not be generated due to an error."

#     return prompt, description


import time

def generate_description_and_prompt(test_case_name, test_case_type, file_path):
    parts = file_path.split('/')
    application_name = parts[-3]
    directory_name = parts[-2]

    # Dynamic action and type based on test case type
    actions = ['Generate', 'Create', 'Build', 'Validate']
    action = random.choice(actions)
    type_phrase = 'keyword-driven' if test_case_type == 'keyword-driven' else 'data-driven'

    # Generate a dynamic prompt
    prompt = f"{action} a robot framework test case for testing {directory_name.replace('_', ' ')} functionality in the {application_name} platform, specifically {test_case_name}."

    description_template = f"Provide a very short description of the test case '{test_case_name}' do"

    # Generate a more general documentation summary
    documentation_template = f"Provide a two to three short sentences general summary of the test case '{test_case_name}', including the main functionality it covers and its importance."

    # Retry logic for API call
    max_retries = 3
    for attempt in range(max_retries):
        try:
            response = openai.chat.completions.create(
                model="gpt-4",
                messages=[{"role": "user", "content": description_template}],
                max_tokens=150
            )
            description = response.choices[0].message.content.strip()
            # Generate the documentation summary
            documentation_response = openai.chat.completions.create(
                model="gpt-4",
                messages=[{"role": "user", "content": documentation_template}],
                max_tokens=150
            )
            documentation = documentation_response.choices[0].message.content.strip()
            return prompt, description, documentation
        except openai.error.APIError as e:
            print(f"API Error generating description or documentation for {test_case_name}: {str(e)}")
            if attempt < max_retries - 1:
                print(f"Retrying... ({attempt + 1}/{max_retries})")
                time.sleep(2)  # Delay before retrying
            else:
                description = f"Description for {test_case_name} could not be generated due to an error."
                documentation = f"Documentation for {test_case_name} could not be generated due to an error."
                return prompt, description, documentation
        except openai.error.RateLimitError:
            print(f"Rate Limit exceeded, waiting before retrying... ({attempt + 1}/{max_retries})")
            time.sleep(30)  # Delay to handle rate limits
        except Exception as e:
            print(f"Unexpected Error generating description or documentation for {test_case_name}: {str(e)}")
            if attempt < max_retries - 1:
                print(f"Retrying... ({attempt + 1}/{max_retries})")
                time.sleep(2)  # Delay before retrying
            else:
                description = f"Description for {test_case_name} could not be generated due to an error."
                documentation = f"Documentation for {test_case_name} could not be generated due to an error."
                return prompt, description, documentation


In [ ]:
def process_data_driven_test_case(file_path):
    test_case_name = os.path.splitext(os.path.basename(file_path))[0]
    test_case_data = pd.read_csv(file_path)

    prompt, description, documentation = generate_description_and_prompt(test_case_name, 'data-driven', file_path)

    # Define application-specific keywords that trigger the "Test Setup" section
    application_keywords = ['PayNOW', 'PremiumPay', 'BillPro', 'BillNOW', 'ClientCentral']

    # Check if the prompt contains any of the application-specific keywords
    setup_needed = any(keyword in prompt for keyword in application_keywords)

    # Start constructing the response
    response = f"""*** Settings ***
Description       {description}
"""

    # Include Test Setup if necessary
    if setup_needed:
        application_name = next((keyword for keyword in application_keywords if keyword in prompt), '')
        response += f"Test Setup       Open {application_name} Application    ${{URL}}\n"

    response += f"""Test Teardown    Close All Browsers
Test Template    Validate {test_case_name}
Resource         ../../../../Config/super.resource
Library          DataDriver    ${{TESTDATA_FOLDER}}    sheet_name={test_case_name}

*** Test Cases ***
    ${{test_case_name}}

*** Keywords ***
Validate {test_case_name}
    [Arguments]    ${{test_case_name}}
    [Description]       {description}
    ${{test_data}}    Read TestData From Excel    ${{TESTDATA_FOLDER}}    ${{test_case_name}}   {test_case_name}
"""

    # Append keywords and arguments
    for _, row in test_case_data.iterrows():
        keyword = row['KeywordName']
        args = [arg for arg in [row.get('Argument1'), row.get('Argument2')] if pd.notna(arg)]
        args_str = '    '.join([f"${{test_data}}[{arg}]" for arg in args])
        response += f"    {keyword}    {args_str}\n"

    return {
        "prompt": prompt,
        "testcase_type": "data-driven",
        "testcase_name": test_case_name,
        "response": response,
        "documentation": documentation
    }


In [ ]:
def process_keyword_driven_test_case(file_path):
    test_case_name = os.path.splitext(os.path.basename(file_path))[0]
    test_case_data = pd.read_csv(file_path)

    prompt, description, documentation = generate_description_and_prompt(test_case_name, 'keyword-driven', file_path)

    # Collect all unique arguments
    all_args = {}
    argument_columns = [col for col in ['Argument1', 'Argument2', 'Argument3', 'Arguments'] if col in test_case_data.columns]

    for col in argument_columns:
        for arg in test_case_data[col].dropna().unique():
            if arg and arg not in all_args:
                all_args[arg.upper()] = f"{arg.lower()}_value"

    # Create the response structure
    response = f"""*** Settings ***
Description          {description}
Library                SeleniumLibrary

*** Variables ***
"""

    # Add dynamic variables based on unique arguments
    for arg, value in all_args.items():
        response += f"${{{arg}}}            {value}\n"

    response += f"""\n*** Test Cases ***
Validate {test_case_name.capitalize()}
    [Description]    {description}
"""

    # Generate test steps for the test case
    for _, row in test_case_data.iterrows():
        keyword = row['KeywordName']
        args = row['Arguments'].split(',') if pd.notna(row['Arguments']) else []
        args_str = '    '.join([f"${{{arg.upper()}}}" for arg in args])
        response += f"    {keyword}    {args_str}\n"

    return {
        "prompt": prompt,
        "testcase_type": "keyword-driven",
        "testcase_name": test_case_name,
        "response": response,
        "documentation": documentation
    }


In [ ]:
def find_csv_files(root_directories):
    csv_files = []
    for root_directory in root_directories:
        for subdir, dirs, files in os.walk(root_directory):
            for file in glob.glob(os.path.join(subdir, '*.csv')):
                csv_files.append(file)
    return csv_files

def process_files(file_paths, process_function):
    results = []
    start_time = time.time()
    for file_path in file_paths:
        try:
            result = process_function(file_path)
            results.append(result)
        except Exception as e:
            print(f"Failed to process {file_path}: {str(e)}")
    end_time = time.time()
    print(f"Processed {len(file_paths)} files in {end_time - start_time:.2f} seconds")
    return results

In [ ]:
# # Root directories for keyword-driven and data-driven test cases
# roots_keyword_driven = [
#     '/content/drive/MyDrive/WORKS/SOFTWARE-DRIVEN-TEST AUTOMATION/dataset & info/SDT UAR Test Cases/ClientCentral_Keywords Ram/ClientCentral_Keywords/API',
#     # ... (other directories)
# ]

# roots_data_driven = [
#     '/content/drive/MyDrive/WORKS/SOFTWARE-DRIVEN-TEST AUTOMATION/dataset & info/SDT UAR Test Cases/PayNOW Test Cases Ramashanker/PayNOW Test Cases/Autopay Testcases',
#     # ... (other directories)
# ]

In [ ]:
# Root directories for keyword-driven and data-driven test cases
roots_keyword_driven = [
    '/content/drive/MyDrive/WORKS/SOFTWARE-DRIVEN-TEST AUTOMATION/dataset & info/SDT UAR Test Cases/ClientCentral_Keywords Ram/ClientCentral_Keywords/API',
    '/content/drive/MyDrive/WORKS/SOFTWARE-DRIVEN-TEST AUTOMATION/dataset & info/SDT UAR Test Cases/ClientCentral_Keywords Ram/ClientCentral_Keywords/Web/CommonKeywordsForAllMerchants',
    '/content/drive/MyDrive/WORKS/SOFTWARE-DRIVEN-TEST AUTOMATION/dataset & info/SDT UAR Test Cases/ClientCentral_Keywords Ram/ClientCentral_Keywords/Web/Omnicorp/Customers',
    '/content/drive/MyDrive/WORKS/SOFTWARE-DRIVEN-TEST AUTOMATION/dataset & info/SDT UAR Test Cases/ClientCentral_Keywords Ram/ClientCentral_Keywords/Web/SterlingCooper/Customers',
    '/content/drive/MyDrive/WORKS/SOFTWARE-DRIVEN-TEST AUTOMATION/dataset & info/SDT UAR Test Cases/ClientCentral_Keywords Ram/ClientCentral_Keywords/Web/SterlingCooper/Dashboard',
    '/content/drive/MyDrive/WORKS/SOFTWARE-DRIVEN-TEST AUTOMATION/dataset & info/SDT UAR Test Cases/ClientCentral_Keywords Ram/ClientCentral_Keywords/Web/SterlingCooper/Invoices',
    '/content/drive/MyDrive/WORKS/SOFTWARE-DRIVEN-TEST AUTOMATION/dataset & info/SDT UAR Test Cases/ClientCentral_Keywords Ram/ClientCentral_Keywords/Web/SterlingCooper/Settings',
    '/content/drive/MyDrive/WORKS/SOFTWARE-DRIVEN-TEST AUTOMATION/dataset & info/SDT UAR Test Cases/ClientCentral_Keywords Ram/ClientCentral_Keywords/Web/SterlingCooper/Transactions',
    '/content/drive/MyDrive/WORKS/SOFTWARE-DRIVEN-TEST AUTOMATION/dataset & info/SDT UAR Test Cases/ClientCentral_Keywords Ram/ClientCentral_Keywords/Web/SterlingCooper/Virtual_Terminal',
    '/content/drive/MyDrive/WORKS/SOFTWARE-DRIVEN-TEST AUTOMATION/dataset & info/SDT UAR Test Cases/PayNOW_Keywords Ramashanker/PayNOW_Keywords/API',
    '/content/drive/MyDrive/WORKS/SOFTWARE-DRIVEN-TEST AUTOMATION/dataset & info/SDT UAR Test Cases/PayNOW_Keywords Ramashanker/PayNOW_Keywords/Web',
    '/content/drive/MyDrive/WORKS/SOFTWARE-DRIVEN-TEST AUTOMATION/dataset & info/SDT UAR Test Cases/PayNOW_Keywords Ramashanker/PayNOW_Keywords/Web/PayNOW',
]

roots_data_driven = [
    '/content/drive/MyDrive/WORKS/SOFTWARE-DRIVEN-TEST AUTOMATION/dataset & info/SDT UAR Test Cases/PayNOW Test Cases Ramashanker/PayNOW Test Cases/Autopay Testcases',
    '/content/drive/MyDrive/WORKS/SOFTWARE-DRIVEN-TEST AUTOMATION/dataset & info/SDT UAR Test Cases/PayNOW Test Cases Ramashanker/PayNOW Test Cases/Omnicorp Testcases',
    '/content/drive/MyDrive/WORKS/SOFTWARE-DRIVEN-TEST AUTOMATION/dataset & info/SDT UAR Test Cases/PayNOW Test Cases Ramashanker/PayNOW Test Cases/SchedulePayment Testcases',
    '/content/drive/MyDrive/WORKS/SOFTWARE-DRIVEN-TEST AUTOMATION/dataset & info/SDT UAR Test Cases/PayNOW Test Cases Ramashanker/PayNOW Test Cases/Sterlingcooper Testcases',
    '/content/drive/MyDrive/WORKS/SOFTWARE-DRIVEN-TEST AUTOMATION/dataset & info/SDT UAR Test Cases/PayNOW Test Cases Ramashanker/PayNOW Test Cases/non integrated Testcases',
    '/content/drive/MyDrive/WORKS/SOFTWARE-DRIVEN-TEST AUTOMATION/dataset & info/SDT UAR Test Cases/PayNOW Datadriver Testcases Ramashanker',
    '/content/drive/MyDrive/WORKS/SOFTWARE-DRIVEN-TEST AUTOMATION/dataset & info/SDT UAR Test Cases/ClientCentral_Testcases Ram/ClientCentral_Testcases/all_transactions_Testcases',
    '/content/drive/MyDrive/WORKS/SOFTWARE-DRIVEN-TEST AUTOMATION/dataset & info/SDT UAR Test Cases/ClientCentral_Testcases Ram/ClientCentral_Testcases/autopay_Testcases',
    '/content/drive/MyDrive/WORKS/SOFTWARE-DRIVEN-TEST AUTOMATION/dataset & info/SDT UAR Test Cases/ClientCentral_Testcases Ram/ClientCentral_Testcases/customers_Testcases',
    '/content/drive/MyDrive/WORKS/SOFTWARE-DRIVEN-TEST AUTOMATION/dataset & info/SDT UAR Test Cases/ClientCentral_Testcases Ram/ClientCentral_Testcases/dashboard_Testcases',
    '/content/drive/MyDrive/WORKS/SOFTWARE-DRIVEN-TEST AUTOMATION/dataset & info/SDT UAR Test Cases/ClientCentral_Testcases Ram/ClientCentral_Testcases/failed_login_attempts_Testcases',
    '/content/drive/MyDrive/WORKS/SOFTWARE-DRIVEN-TEST AUTOMATION/dataset & info/SDT UAR Test Cases/ClientCentral_Testcases Ram/ClientCentral_Testcases/invoices_Testcases',
    '/content/drive/MyDrive/WORKS/SOFTWARE-DRIVEN-TEST AUTOMATION/dataset & info/SDT UAR Test Cases/ClientCentral_Testcases Ram/ClientCentral_Testcases/payment_status_Testcases',
    '/content/drive/MyDrive/WORKS/SOFTWARE-DRIVEN-TEST AUTOMATION/dataset & info/SDT UAR Test Cases/ClientCentral_Testcases Ram/ClientCentral_Testcases/scheduled_payments_Testcases',
    '/content/drive/MyDrive/WORKS/SOFTWARE-DRIVEN-TEST AUTOMATION/dataset & info/SDT UAR Test Cases/ClientCentral_Testcases Ram/ClientCentral_Testcases/settings_Testcases',
    '/content/drive/MyDrive/WORKS/SOFTWARE-DRIVEN-TEST AUTOMATION/dataset & info/SDT UAR Test Cases/ClientCentral_Testcases Ram/ClientCentral_Testcases/tokens_Testcases',
    '/content/drive/MyDrive/WORKS/SOFTWARE-DRIVEN-TEST AUTOMATION/dataset & info/SDT UAR Test Cases/ClientCentral_Testcases Ram/ClientCentral_Testcases/virtual_terminal_Testcases'
]

In [ ]:
# Collect all relevant CSV files
keyword_driven_files = find_csv_files(roots_keyword_driven)
data_driven_files = find_csv_files(roots_data_driven)


In [ ]:
print(f"Found {len(keyword_driven_files)} keyword-driven test cases")
print(f"Found {len(data_driven_files)} data-driven test cases")

Found 38 keyword-driven test cases
Found 343 data-driven test cases


In [ ]:
# Process keyword-driven and data-driven test cases
refined_KD_data = process_files(keyword_driven_files, process_keyword_driven_test_case)
refined_DD_data = process_files(data_driven_files, process_data_driven_test_case)

Processed 38 files in 233.20 seconds
Failed to process /content/drive/MyDrive/WORKS/SOFTWARE-DRIVEN-TEST AUTOMATION/dataset & info/SDT UAR Test Cases/PayNOW Test Cases Ramashanker/PayNOW Test Cases/Sterlingcooper Testcases/TC_50 Validate Customer Is Able To Cancel Login With Invoice Pop-Up.csv: 'utf-8' codec can't decode byte 0x91 in position 102: invalid start byte
Processed 343 files in 2073.33 seconds


In [ ]:
# Convert to DataFrame and Save refined data to CSV and Excel
keydriven_df = pd.DataFrame(refined_KD_data)
ddriven_df = pd.DataFrame(refined_DD_data)

In [ ]:
# keydriven_df.head()

In [ ]:
output_path = '/content/drive/MyDrive/WORKS/SOFTWARE-DRIVEN-TEST AUTOMATION/dataset & info/SDT UAR Test Cases/'
keydriven_df.to_csv(output_path + 'keyDriven_testcases.csv', index=False)
ddriven_df.to_csv(output_path + 'Data_driven_testcases.csv', index=False)
keydriven_df.to_excel(output_path + 'keyDriven_testcases.xlsx', index=False)
ddriven_df.to_excel(output_path + 'Data_driven_testcases.xlsx', index=False)

print("Processing completed. Files saved successfully.")

Processing completed. Files saved successfully.
